In [2]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# URL 설정
url = "https://m.search.naver.com/search.naver?ssc=tab.m_news.all&where=m_news&sm=mtb_jum&query=s%26p500"

# HTTP 요청
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
soup

search_result = soup.select_one('#news_result_list')
search_result

news_links = search_result.select('.bx > .news_wrap > a')
print(news_links)


<ul class="list_news _infinite_list" data-img-mutation="" id="news_result_list"> <li class="bx"> <div class="news_wrap"> <div class="news_info"> <div class="api_save_group _keep_wrap"> <a class="btn_save _keep_trigger" data-url="https://n.news.naver.com/article/366/0001030488?sid=104" href="#" onclick="tCR('a=nws*h.kep&amp;r=1&amp;i=88127058_000000000000000001030488&amp;u=javascript'); return false;" role="button"><i class="spnew ico_save">문서 저장하기</i></a> <div class="api_ly_save _keep_save_layer"> <a class="spnew_af item item_save _keep_save" data-cr-off="a=nws*h.kepoff&amp;r=1&amp;i=88127058_000000000000000001030488&amp;u=javascript" data-cr-on="a=nws*h.kepon&amp;r=1&amp;i=88127058_000000000000000001030488&amp;u=javascript" href="#" role="button">Keep에 저장</a> <a class="spnew_af item item_quick" href="https://m.keep.naver.com/" onclick="return goOtherCR(this,'a=nws*h.kephome&amp;r=1&amp;i=88127058_000000000000000001030488&amp;u='+urlencode(this.href));">Keep 바로가기</a> </div> </div><div 

In [28]:
news_text = search_result.select('.bx > .news_wrap > div')
print(news_text)

for news_link, news_text in zip(news_links, news_text):
    print(news_link.text)
    print(news_link['href'])
    print(news_text.text)
    print('===')

[<div class="news_info"> <div class="api_save_group _keep_wrap"> <a class="btn_save _keep_trigger" data-url="https://n.news.naver.com/article/366/0001030488?sid=104" href="#" onclick="tCR('a=nws*h.kep&amp;r=1&amp;i=88127058_000000000000000001030488&amp;u=javascript'); return false;" role="button"><i class="spnew ico_save">문서 저장하기</i></a> <div class="api_ly_save _keep_save_layer"> <a class="spnew_af item item_save _keep_save" data-cr-off="a=nws*h.kepoff&amp;r=1&amp;i=88127058_000000000000000001030488&amp;u=javascript" data-cr-on="a=nws*h.kepon&amp;r=1&amp;i=88127058_000000000000000001030488&amp;u=javascript" href="#" role="button">Keep에 저장</a> <a class="spnew_af item item_quick" href="https://m.keep.naver.com/" onclick="return goOtherCR(this,'a=nws*h.kephome&amp;r=1&amp;i=88127058_000000000000000001030488&amp;u='+urlencode(this.href));">Keep 바로가기</a> </div> </div><div class="info_group"> <a class="info press" href="https://media.naver.com/press/366" onclick="return goOtherCR(this, 'a=nw

In [52]:
# URL 설정
url = "https://news.kbs.co.kr/news/pc/search/search.do?query=s&p%20500&sortType=date&filter=1&page=1&rpage=0"

# HTTP 요청
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
soup

# 기사 URL과 제목 크롤링
articles = soup.select_one('#search-result-tab')

articles

# news_links = articles.select('.tab-contents-list date > .SearchResult-searchResultContent > a')
# print(news_links)

<ul class="tab-contents" id="search-result-tab">
<li class="tab-contents-list score">
<div class="box-contents" style="display:none;">
</div>
<div class="common-pagination" style="display:none;">
<button class="double-prev-button" type="button"></button>
<button class="prev-button" type="button"></button>
<ul class="number-buttons">
<li class="number-buttons-list">
<button class="number-button" type="button"></button>
</li>
</ul>
<button class="next-button" type="button"></button>
<button class="double-next-button" type="button"></button>
</div>
</li>
<li class="tab-contents-list date">
<div class="box-contents" style="display:none;"></div>
<div class="common-pagination">
<button class="double-prev-button" type="button"></button>
<button class="prev-button" type="button"></button>
<ul class="number-buttons">
</ul>
<button class="next-button" type="button"></button>
<button class="double-next-button" type="button"></button>
</div>
</li>
</ul>

In [66]:
# import package
from bs4 import BeautifulSoup
import requests
import urllib
import re
import pandas as pd
from newspaper import Article
from selenium import webdriver

keyword = 'Fed'
page_num = 0  # 0, 10, 20...
news_df = pd.DataFrame()

# selenium headless mode
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)

In [67]:
while True:
    # Google 뉴스 크롤링
    url = f'https://www.google.com/search?q={keyword}&sca_esv=1814fa2a4600643d&tbas=0&tbs=qdr:m&tbm=nws&ei=rE3pZeLxNeHX1e8PpdOcMA&start={page_num}&sa=N&ved=2ahUKEwji9-zrsuGEAxXha_UHHaUpBwYQ8tMDegQIBBAE&biw=2560&bih=1313&dpr=1'
    req = requests.get(url)
    content = req.content
    soup = BeautifulSoup(content, 'html.parser')

    # 마지막 페이지 확인
    if not soup.select('div.BNeawe.vvjwJb'):
        break

    title_list = [t.text for t in soup.select('div.BNeawe.vvjwJb')]  # 제목 리스트
    url_list = []

    # URL 추출
    for u in soup.select('a'):
        for t in title_list:
            if t in u.text:
                temp_url = urllib.parse.unquote(u['href'])
                temp_url = re.findall('http\S+&sa', temp_url)[0][:-3]
                url_list.append(temp_url)

    # 기사 크롤링
    for ind, news_url in enumerate(url_list):
        try:
            article = Article(url=news_url)
            article.download()
            article.parse()
            news_article = article.text
        except:  # SSL 오류 처리
            driver.get(news_url)
            article.download(input_html=driver.page_source)
            article.parse()
            news_article = article.text

        # 인덱스가 범위를 벗어나지 않는지 확인
        if ind < len(title_list):
            news_df = pd.concat([news_df, pd.DataFrame([[title_list[ind], news_article, news_url]])])
        else:
            print(f"Index {ind} is out of range for title_list")

    news_df[0] = news_df[0].apply(lambda x: re.sub('\s+', ' ', x))
    news_df = news_df.reset_index(drop=True)

    page_num += 10
    news_df.to_excel('news_crawling.xlsx', index=False)

news_df.columns = ['제목', '본문', 'URL']


<>:20: SyntaxWarning: invalid escape sequence '\S'
<>:42: SyntaxWarning: invalid escape sequence '\s'
<>:20: SyntaxWarning: invalid escape sequence '\S'
<>:42: SyntaxWarning: invalid escape sequence '\s'
C:\Users\KimDongyoung\AppData\Local\Temp\ipykernel_16596\4236816558.py:20: SyntaxWarning: invalid escape sequence '\S'
  temp_url = re.findall('http\S+&sa', temp_url)[0][:-3]
C:\Users\KimDongyoung\AppData\Local\Temp\ipykernel_16596\4236816558.py:42: SyntaxWarning: invalid escape sequence '\s'
  news_df[0] = news_df[0].apply(lambda x: re.sub('\s+', ' ', x))


In [68]:
news_df

,제목,본문,URL
0,"파월 ""Fed 의장 해임 불가능…재정적자 해결해야"" [Fed워치]","파월 ""美 대선 결과, 정책 결정에 영향 없어""\n\n대통령이 Fed 의장 해임 할...",https://www.hankyung.com/article/202411088301i
1,"""Fed 11월 금리 인하 전망""…트럼프 당선하면 어떻게 되나","[美 선택 2024]\n\nJP모건 ""트럼프 당선 시 금리인하 중단""\n\n5일(현...",https://cm.asiae.co.kr/article/202411041349182...
2,"美 기준금리 0.25%p 인하…파월 ""Fed는 공공임무 수행"" [Fed워치]","제롬 파월 Fed 의장 모두발언\n\n""노동시장 조건 여전히 견고""\n\n사진=로이...",https://www.hankyung.com/article/202411088272i
3,"美 Fed 베이지북 ""인플레이션 완화됐지만 기업 영업이익 줄어""",12개 지역 중 2곳만 성장\n\n인플레이션 압박 완화했지만 꽁꽁 닫힌 지갑\n\n...,https://www.hankyung.com/article/202410245514i
4,Fed’s decision to continue rate cut cycle may ...,Necessary Cookies\n\nThis cookie is used to di...,https://www.aa.com.tr/en/economy/fed-s-decisio...
...,...,...,...
555,Why the Fed will likely cut rates in December ...,,https://www.marketwatch.com/story/why-the-fed-...
556,Thousands ordered to evacuate as powerful wind...,"CAMARILLO, Calif. (AP) — California was lashed...",https://www.yahoo.com/news/powerful-winds-low-...
557,"Fed Cuts Rates, Powell Charts 'Middle Path'; S...",Access to this page has been denied because we...,https://www.investors.com/news/economy/federal...
558,Ethereum Price Forecast: ETH could see fresh d...,"Ethereum price today: $2,960\n\nEthereum could...",https://www.fxstreet.com/cryptocurrencies/news...


In [70]:
# Selenium에서 사용할 기본 모듈들을 불러옵니다.
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


# 엑셀 파일을 다루기 위한 openpyxl 패키지를 불러옵니다.
from openpyxl import Workbook, load_workbook

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [72]:
# URL 컬럼의 값을 리스트로 변환
url_list_from_df = news_df['URL'].tolist()
len(url_list_from_df)

560

In [ ]:
def extract_article_content(url):
    # 웹 페이지 요청
    response = requests.get(url)

    # 페이지 내용 파싱
    soup = BeautifulSoup(response.content, 'html.parser')

    # 기사 제목 추출
    title_element = soup.find('h1')
    title = title_element.get_text() if title_element else "Title not found"

    # 본문 텍스트 추출
    article_body_elements = soup.find_all('div', class_='paragraph')
    if not article_body_elements:
        article_body_elements = soup.find_all('div', class_='zn-body__paragraph')
    if not article_body_elements:
        article_body_elements = soup.find_all('p')  # 다른 가능한 본문 태그 시도

    # 본문 텍스트를 합침
    article_text = ' '.join([paragraph.get_text() for paragraph in article_body_elements])

    # 본문을 줄 단위로 분리
    lines = article_text.splitlines()

    # 앞 18줄 제거하고 마지막 2줄도 제거
    cleaned_article_text = ' '.join(lines[18:-2]).replace('\n', ' ')  # 18줄 이후의 텍스트에서 마지막 2줄 제외 및 줄바꿈 제거

    return title, cleaned_article_text.strip()  # 추가 공백 제거

# 기사 내용을 저장할 리스트
articles = []

# URL 리스트를 반복하여 기사 내용 추출
for url in url_list_from_df:
    title, content = extract_article_content(url)
    articles.append({
        'url': url,
        'title': title,
        'content': content
    })

# 데이터프레임 생성
df = pd.DataFrame(articles)

# 데이터프레임 출력
df